# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Li  ->  S. Li  |  ['S. Li']
C. Bailer-Jones  ->  C. Bailer-Jones  |  ['C. Bailer-Jones']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
J. He  ->  J. He  |  ['J. He']
Arxiv has 58 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2311.00186


extracting tarball to tmp_2311.00186...

 done.
Retrieving document from  https://arxiv.org/e-print/2311.00374


extracting tarball to tmp_2311.00374...

...

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure plots/distance_prior_median_skyplot
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure plots/distance_prior_median_skyplot as tmp_2311.00374/./plots/distance_prior_median_skyplot.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 29 bibliographic references in tmp_2311.00374/gedr3_distances_velocities.bbl.
Retrieving document from  https://arxiv.org/e-print/2311.00477


extracting tarball to tmp_2311.00477...

 done.
Retrieving document from  https://arxiv.org/e-print/2311.00615


/tmp/ipykernel_2154/3009462391.py:49: LatexWarning: 2311.00615 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.00374-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.00374) | **Estimating distances from parallaxes. VI: A method for inferring  distances and transverse velocities from parallaxes and proper motions  demonstrated on Gaia Data Release 3**  |
|| <mark>C. Bailer-Jones</mark> |
|*Appeared on*| *2023-11-02*|
|*Comments*| *Accepted to AJ. Details on the velocity prior for each HEALpixel are available at this https URL*|
|**Abstract**| The accuracy of stellar distances inferred purely from parallaxes degrades rapidly with distance. Proper motion measurements, when combined with some idea of typical velocities, provide independent information on stellar distances. Here I build a direction- and distance-dependent model of the distribution of stellar velocities in the Galaxy, then use this together with parallaxes and proper motions to infer kinegeometric distances and transverse velocities for stars in Gaia DR3. Using noisy simulations I assess the performance of the method and compare its accuracy to purely parallax-based (geometric) distances. Over the whole Gaia catalogue, kinegeometric distances are on average 1.25 times more accurate than geometric ones. This average masks a large variation in the relative performance, however. Kinegeometric distances are considerably better than geometric ones beyond several kpc, for example. On average, kinegeometric distances can be measured to an accuracy of 19% and velocities (sqrt[vra^2 + vdec^2]) to 16 km/s (median absolute deviations). In Gaia DR3, kinegeometric distances are smaller than geometric ones on average for distant stars, but the pattern is more complex in the bulge and disk. With the much more accurate proper motions expected in Gaia DR5, a further improvement in the distance accuracy by a factor of (only) 1.35 on average is predicted (with kinegeometric distances still 1.25 times more accurate than geometric ones). The improvement from proper motions is limited by the width of the velocity prior, in a way that the improvement from better parallaxes is not limited by the width of the distance prior. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.00186-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.00186) | **Image Restoration with Point Spread Function Regularization and Active  Learning**  |
|| P. Jia, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-11-02*|
|*Comments*| *To be published in the MNRAS*|
|**Abstract**| Large-scale astronomical surveys can capture numerous images of celestial objects, including galaxies and nebulae. Analysing and processing these images can reveal intricate internal structures of these objects, allowing researchers to conduct comprehensive studies on their morphology, evolution, and physical properties. However, varying noise levels and point spread functions can hamper the accuracy and efficiency of information extraction from these images. To mitigate these effects, we propose a novel image restoration algorithm that connects a deep learning-based restoration algorithm with a high-fidelity telescope simulator. During the training stage, the simulator generates images with different levels of blur and noise to train the neural network based on the quality of restored images. After training, the neural network can directly restore images obtained by the telescope, as represented by the simulator. We have tested the algorithm using real and simulated observation data and have found that it effectively enhances fine structures in blurry images and increases the quality of observation images. This algorithm can be applied to large-scale sky survey data, such as data obtained by LSST, Euclid, and CSST, to further improve the accuracy and efficiency of information extraction, promoting advances in the field of astronomical research. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.00477-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.00477) | **Massive star-formation in the hub-filament system of RCW 117**  |
|| A. Seshadri, S. Vig, <mark>S. Ghosh</mark>, D. Ojha |
|*Appeared on*| *2023-11-02*|
|*Comments*| *15 pages, 11 figures, 6 tables, Accepted for publication in Monthly Notices of the Royal Astronomical Society (MNRAS)*|
|**Abstract**| We present a multiwavelength investigation of the hub-filament system RCW 117 (IRAS 17059-4132), which shows intricate filamentary features in the far-infrared, mapped using Herschel images. We obtain the column density and dust temperature maps for the region using the Herschel images, and identify 88 cores and 12 filaments from the column density map of the region ($18'\times18'$). We employ the ThrUMMS $^{13}$CO (J=1-0) data for probing the kinematics in RCW 117, and find velocity gradients ($\sim 0.3-1$ km s$^{-1}$ pc$^{-1}$) with hints of matter inflow along the filamentary structures. Ionised gas emission from the associated HII region is examined using the Giant Metrewave Radio Telescope (GMRT) at 610 and 1280 MHz, and is found to be of extent $5 \times 3$ pc$^2$ with intensity being brightest towards the hub. We estimate the peak electron density towards the hub to be $\sim 750$ cm$^{-3}$. Thirty four Class 0/I young stellar objects (YSOs) have been identified in the region using the Spitzer GLIMPSE colour-colour diagram, with many lying along the filamentary structures. Based on the (i) presence of filamentary structures, (ii) distribution of cores across the region, with $\sim39$% found along the filamentary structures, (iii) massive star-formation tracers in the hub, and (iv) the kinematics, we believe that global hierarchical collapse can plausibly explain the observed features in RCW 117. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2311.00615-b31b1b.svg)](https://arxiv.org/abs/arXiv:2311.00615) | **2023 Astrophotonics Roadmap: pathways to realizing multi-functional  integrated astrophotonic instruments**  |
|| N. Jovanovic, et al. -- incl., <mark>J. He</mark> |
|*Appeared on*| *2023-11-02*|
|*Comments*| *191 pages, 47 figures. This is the version of the article before peer review or editing, as submitted by an author to J. Phys. Photonics. IOP Publishing Ltd is not responsible for any errors or omissions in this version of the manuscript or any version derived from it. The Version of Record is available online at this https URL*|
|**Abstract**| Photonics offer numerous functionalities that can be used to realize astrophotonic instruments. The most spectacular example to date is the ESO Gravity instrument at the Very Large Telescope in Chile. Integrated astrophotonic devices stand to offer critical advantages for instrument development, including extreme miniaturization, as well as integration, superior thermal and mechanical stabilization owing to the small footprint, and high replicability offering cost savings. Numerous astrophotonic technologies have been developed to address shortcomings of conventional instruments to date, including for example the development of photonic lanterns, complex aperiodic fiber Bragg gratings, complex beam combiners to enable long baseline interferometry, and laser frequency combs for high precision spectral calibration of spectrometers. Despite these successes, the facility implementation of photonic solutions in astronomical instrumentation is currently limited because of (1) low throughputs from coupling to fibers, coupling fibers to chips, propagation and bend losses, device losses, etc, (2) difficulties with scaling to large channel count devices needed for large bandwidths and high resolutions, and (3) efficient integration of photonics with detectors, to name a few. In this roadmap, we identify 24 areas that need further development. We outline the challenges and advances needed across those areas covering design tools, simulation capabilities, fabrication processes, the need for entirely new components, integration and hybridization and the characterization of devices. To realize these advances the astrophotonics community will have to work cooperatively with industrial partners who have more advanced manufacturing capabilities. With the advances described herein, multi-functional instruments will be realized leading to novel observing capabilities for both ground and space platforms. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2311.00374.md
    + _build/html/tmp_2311.00374/./plots/veltotmeanprior_0pc_skyplot.png
    + _build/html/tmp_2311.00374/./plots/veltotmeanprior_1000pc_skyplot.png
    + _build/html/tmp_2311.00374/./plots/veltotmeanprior_2000pc_skyplot.png
    + _build/html/tmp_2311.00374/./plots/veltotmeanprior_4000pc_skyplot.png
    + _build/html/tmp_2311.00374/./plots/veltotmeanprior_6000pc_skyplot.png
    + _build/html/tmp_2311.00374/./plots/veltotmeanprior_10000pc_skyplot.png
    + _build/html/tmp_2311.00374/./plots/results_mock_cfs_rMedKinogeo_bias_vs_distance.png
    + _build/html/tmp_2311.00374/./plots/results_mock_cfs_rMedKinogeo_scatter_vs_distance.png
    + _build/html/tmp_2311.00374/./plots/results_cns_rMedGeo_skyplot.png
    + _build/html/tmp_2311.00374/./plots/results_cns_rMedKinogeo_skyplot.png
    + _build/html/tmp_2311.00374/./plots/results_cns_rMedGeo_rMedKinogeo_fracdiff_skyplot.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sourceid}{\texttt{source\_id}}$
$\newcommand{\deriv}{\ensuremath{\mathrm{d}}}$
$\newcommand{\given}{\ensuremath{\hspace{0.05em}\mid\hspace{0.05em}}}$
$\newcommand{\logten}{\ensuremath{\log_{\rm 10}}}$
$\newcommand{\deriv}{\ensuremath{\mathrm{d}}}$
$\newcommand{\given}{\ensuremath{\hspace{0.01em}\mid\hspace{0.01em}}}$
$\newcommand{\logten}{\ensuremath{\log_{\rm 10}}}$
$\newcommand{\gaia}{Gaia}$
$\newcommand{\gdr}[1]{GDR{#1}}$
$\newcommand{\gedr}[1]{GeDR{#1}}$
$\newcommand{\mock}{GeDR3mock}$
$\newcommand{\release}{GDR3}$
$\newcommand{\gmag}{\ensuremath{G}}$
$\newcommand{\parallax}{\ensuremath{\varpi}}$
$\newcommand{\parallaxzp}{\ensuremath{\varpi_{\rm zp}}}$
$\newcommand{\fpu}{\ensuremath{\sigparallax/\parallax}}$
$\newcommand{\propm}{\ensuremath{\mu}}$
$\newcommand{\pmra}{\ensuremath{\mu_{\alpha*}}}$
$\newcommand{\pmdec}{\ensuremath{\mu_{\delta}}}$
$\newcommand{\sigparallax}{\ensuremath{\sigma_{\varpi}}}$
$\newcommand{\cov}{\ensuremath{\Sigma}}$
$\newcommand{\glon}{\ensuremath{l}}$
$\newcommand{\glat}{\ensuremath{b}}$
$\newcommand{\hp}{\ensuremath{p}}$
$\newcommand{\dist}{\ensuremath{r}}$
$\newcommand{\rest}{\ensuremath{\dist_{\rm est}}}$
$\newcommand{\rlo}{\ensuremath{\dist_{\rm lo}}}$
$\newcommand{\rhi}{\ensuremath{\dist_{\rm hi}}}$
$\newcommand{\rmode}{\ensuremath{\dist_{\rm mode}}}$
$\newcommand{\rmed}{\ensuremath{\dist_{\rm med}}}$
$\newcommand{\rEDSDmode}{\ensuremath{\dist^{\rm EDSD}_{\rm mode}}}$
$\newcommand{\rinit}{\ensuremath{\dist_{\rm init}}}$
$\newcommand{\rlen}{\ensuremath{L}}$
$\newcommand{\myalpha}{\ensuremath{\alpha}}$
$\newcommand{\mybeta}{\ensuremath{\beta}}$
$\newcommand{\vra}{\ensuremath{v_{\alpha*}}}$
$\newcommand{\vdec}{\ensuremath{v_{\delta}}}$
$\newcommand{◦ee}{\ensuremath{^\circ}}$
$\newcommand{\kms}{\ensuremath{\mathrm{km} \mathrm{s}^{-1}}}$
$\newcommand{\pc}{\ensuremath{\mathrm{pc}}}$
$\newcommand{\kpc}{\ensuremath{\mathrm{kpc}}}$
$\newcommand{\mas}{\ensuremath{\mathrm{mas}}}$
$\newcommand{\maspyr}{\ensuremath{\mathrm{mas \mathrm{yr}^{-1}}}}$
$\newcommand{\red}{\textcolor{red}}$</div>



<div id="title">

# Estimating distances from parallaxes. VI:$\A method for inferring distances and transverse velocities from parallaxes and proper motions$\\demonstrated on Gaia Data Release 3

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2311.00374-b31b1b.svg)](https://arxiv.org/abs/2311.00374)<mark>Appeared on: 2023-11-02</mark> -  _Accepted to AJ. Details on the velocity prior for each HEALpixel are available at this https URL_

</div>
<div id="authors">

<mark>C. Bailer-Jones</mark>

</div>
<div id="abstract">

**Abstract:** The accuracy of stellar distances inferred purely from parallaxes degrades rapidly with distance.Proper motion measurements, when combined with some idea of typical velocities, provide independent information on stellar distances.  Here I build a direction- and distance-dependent model of the distribution of stellar velocities in the Galaxy, then use this together with parallaxes and proper motions to infer kinegeometric distances and transverse velocities for stars in $\gaia$ DR3.  Using noisy simulations I assess the performance of the method and compare its accuracy to purely parallax-based (geometric) distances. Over the whole $\gaia$ catalogue, kinegeometric distances are on average 1.25 times  more accurate than geometric ones. This average masks a large variation in the relative performance, however. Kinegeometric distances are considerably better than geometric ones  beyond several $\kpc$ , for example.    On average, kinegeometric distances can be measured to an accuracy of 19 \% and velocities( $\sqrt{\vra^2 + \vdec^2}$ )  to 16 $\kms$ (median absolute deviations).In $\gaia$ DR3, kinegeometric distances are smaller than geometric ones on average for distant stars, but the pattern is more complex in the bulge and disk.With the much more accurate proper motions expected in $\gaia$ DR5, a further improvement in the distance accuracy by a factor of (only) 1.35 on average is predicted(with kinegeometric distances still 1.25 times more accurate than geometric ones).The improvement attained from proper motions is limited by the width of the velocity prior, in a way that the improvement from better parallaxes is not limited by the width of the distance prior.

</div>

<div id="div_fig1">

<img src="tmp_2311.00374/./plots/veltotmeanprior_0pc_skyplot.png" alt="Fig13.1" width="16%"/><img src="tmp_2311.00374/./plots/veltotmeanprior_1000pc_skyplot.png" alt="Fig13.2" width="16%"/><img src="tmp_2311.00374/./plots/veltotmeanprior_2000pc_skyplot.png" alt="Fig13.3" width="16%"/><img src="tmp_2311.00374/./plots/veltotmeanprior_4000pc_skyplot.png" alt="Fig13.4" width="16%"/><img src="tmp_2311.00374/./plots/veltotmeanprior_6000pc_skyplot.png" alt="Fig13.5" width="16%"/><img src="tmp_2311.00374/./plots/veltotmeanprior_10000pc_skyplot.png" alt="Fig13.6" width="16%"/>

**Figure 13. -** Variation of the mean of the velocity prior over the Galactic sky at six different distances. The quantity shown
    is $\sqrt{$\vra$^2 + $\vdec$^2}$, where $\vra$ and $\vdec$ are the means of the two components of the velocity prior at the specified distance in each HEALpixel. In each distance slice the colour bar covers the full range of velocities plotted, and is different for each slice.
 (*fig:veltotmeanprior_skyplot*)

</div>
<div id="div_fig2">

<img src="tmp_2311.00374/./plots/results_mock_cfs_rMedKinogeo_bias_vs_distance.png" alt="Fig16.1" width="50%"/><img src="tmp_2311.00374/./plots/results_mock_cfs_rMedKinogeo_scatter_vs_distance.png" alt="Fig16.2" width="50%"/>

**Figure 16. -** Performance of the kinegeometric distance estimate (median of the posterior) as a function of distance for the constant fraction sample in the mock catalogue. The left panel shows the fractional bias in the estimates, the right panel the fractional scatter. The colour scale shows the density of stars on a log scale. The solid orange line shows the median bias or scatter at each distance, the dotted lines are the 5th and 95th percentiles. The blue lines show the same metrics for the geometric distances for comparison.
 (*fig:results_mock_rMedKinogeo_bias_scatter_vs_distance*)

</div>
<div id="div_fig3">

<img src="tmp_2311.00374/./plots/results_cns_rMedGeo_skyplot.png" alt="Fig5.1" width="33%"/><img src="tmp_2311.00374/./plots/results_cns_rMedKinogeo_skyplot.png" alt="Fig5.2" width="33%"/><img src="tmp_2311.00374/./plots/results_cns_rMedGeo_rMedKinogeo_fracdiff_skyplot.png" alt="Fig5.3" width="33%"/>

**Figure 5. -** Median distance per HEALpixel for the constant number sample in $\release$: Geometric (top) and kinegeometric (middle).
  The linear colour bar spans the full common range. The bottom panel shows
  the median fractional differences between these (computed on the individual stars). It
uses a bilinear colour bar with white at zero and diverges on different linear scales to the maximum negative and positive differences.
 (*fig:results_rMedGeo_rMedKinogeo_skyplot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2311.00374"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

315  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
